In [111]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from datetime import datetime
import re
import string
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [112]:
model = pickle.load(open('./old/modelSVC.pickle','rb'))
tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))

/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_4569/3135097857.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  model = pickle.load(open('./old/modelSVC.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_4569/3135097857.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9/s

In [113]:
def scraperKey(keyword,date_since,date_until):
    query = keyword+" lang:id until:"+str(date_until)+" since:"+str(date_since)
    #print(query)
    #print(datetime.now())
    #print("Sedang Mengumpulkan Data Twitter...")
    tweets = []
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.hashtags, tweet.likeCount, tweet.retweetCount, tweet.replyCount])
        
        
        
    df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'Hashtags', 'Like', 'Retweet', 'Reply'])
    return df


In [114]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = text.encode("ascii", "ignore").decode() #remove emojis

    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    text = [w for w in text if not w in listStopwords]
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    #factory = StemmerFactory()
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

alay_dict = pd.read_csv('colloquial-indonesian-lexicon.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

def normalize_alay(text):
 text = ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])
 text = re.sub(' +', ' ', text)
 return text



In [130]:
df = scraperKey('pemilu','2022-12-15','2022-12-18')

In [131]:
df.head()

,Date,User,Tweet,Hashtags,Like,Retweet,Reply
0,2022-12-17 23:59:55+00:00,syahidun58,@HelmiFelis_ Beliau tidak mau berseteru dengan...,None,0,0,0
1,2022-12-17 23:58:49+00:00,PKS_Bojongmangu,"Kembali Dapat Nomor Urut 8 di Pemilu 2024, PKS...",None,0,0,0
2,2022-12-17 23:58:35+00:00,PKS_Bojongmangu,PKS Tekankan Pemilu 2024 Harus Jujur dan Adil ...,None,0,0,0
3,2022-12-17 23:58:33+00:00,PKS_Bojongmangu,"Dapat Nomor Urut 8, Presiden PKS: Mari Kita Ka...",None,0,0,0
4,2022-12-17 23:57:58+00:00,dedypr_,"Bukanya knp knp ya, stiap menjelang pemilu lih...",None,0,0,0


In [132]:
if df.empty:
    print("Tidak ada data atau query keliru!")
else:
    df = df.drop_duplicates(subset=['Tweet'])
    dataset = df
    #print(dataset)
    print("Sedang Membersihkan Data Twitter...")
    #gabungin hashtags
    # dataset['Hashtags'] = '-'.join(dataset['Hashtags'])
    print(datetime.now())
    print('cleaning text')
    dataset['text_clean'] = dataset['Tweet'].apply(cleaningText)
    print(datetime.now())
    print('casefolding text')
    dataset['text_clean'] = dataset['text_clean'].apply(casefoldingText)
    dataset['text_clean'] = dataset['text_clean'].apply(normalize_alay)
    # dataset.drop(['Tweet'], axis = 1, inplace = True)
    print(datetime.now())
    print('tokenizing text')
    dataset['text_preprocessed'] = dataset['text_clean'].apply(tokenizingText)
    print(datetime.now())
    print("filtering text")
    dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(filteringText)
    print(datetime.now())
    print('stemming text')
    dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(stemmingText)
    print("Selesai Membersihkan Data Twitter :)")
    print(datetime.now())
    print(dataset)
    cleanData = dataset

        ###################################
        ###      SENTIMENT ANALYSIS     ###
        ###################################

    print("Sedang Menganalisis Sentimen Publik...")
    print(datetime.now())
    # Make text preprocessed (tokenized) to untokenized with toSentence Function
    X = cleanData['text_preprocessed'].apply(toSentence)
    X = tfidf_vectorizer.transform(X.values)
    
    # e = X.toarray()
    # #rumus mean
    # n = 0
    # mean = []
    # for i in e:
    #     a = sum(i.tolist())/5000
    #     mean.append(a)

    # #standar deviasi
    # n = 0
    # stdev = []
    # for i in e:
    #     s = np.std(i.tolist())
    #     stdev.append(s)

    # #max value
    # n = 0
    # maks = []
    # for i in e:
    #     a = max(i.tolist())
    #     maks.append(a)

    # #sum
    # n = 0
    # summ = []
    # for i in e:
    #     a = sum(i.tolist())
    #     summ.append(a)

    # #count feature
    # n = 0
    # count = []
    # for i in e:
    #     a = sum(map(lambda x : x != 0, i.tolist()))
    #     count.append(a)

    # extra_X = np.column_stack((e, np.array(mean), np.array(stdev), np.array(maks), np.array(summ), np.array(count)))

    # from scipy import sparse
    # extra_X = sparse.csr_matrix(extra_X)

    y_pred = model.predict(X)
    cleanData['Result Prediction'] = y_pred

    polarity_decode = {0 : 'Negative', 1 : 'Neutral', 2 : 'Positive'}
    cleanData['Result Prediction'] = cleanData['Result Prediction'].map(polarity_decode)
    print("Selesai Menganalisis Sentimen Publik :)")
    print(datetime.now())
    print(cleanData[['text_clean','Result Prediction']])
    #print('OK')
    print(cleanData)
    cleanData['Date'] = pd.to_datetime(cleanData['Date']).dt.date
    cleanData['Date']=cleanData['Date'].astype(str)
        
    print(cleanData.dtypes)
    # data_dict = cleanData.to_dict(orient='records')
    # print(data_dict)

Sedang Membersihkan Data Twitter...
2023-01-10 13:42:32.588856
cleaning text
2023-01-10 13:42:32.688312
casefolding text
2023-01-10 13:42:32.764661
tokenizing text
2023-01-10 13:42:33.635552
filtering text
2023-01-10 13:42:35.874592
stemming text
Selesai Membersihkan Data Twitter :)
2023-01-10 13:42:37.518494
                          Date             User  \
0    2022-12-17 23:59:55+00:00       syahidun58   
1    2022-12-17 23:58:49+00:00  PKS_Bojongmangu   
2    2022-12-17 23:58:35+00:00  PKS_Bojongmangu   
3    2022-12-17 23:58:33+00:00  PKS_Bojongmangu   
4    2022-12-17 23:57:58+00:00          dedypr_   
...                        ...              ...   
9598 2022-12-15 00:00:11+00:00         ekodj373   
9599 2022-12-15 00:00:06+00:00    toServetoGive   
9600 2022-12-15 00:00:02+00:00        adekaka01   
9601 2022-12-15 00:00:01+00:00   liputan6dotcom   
9602 2022-12-15 00:00:01+00:00     WartaEkonomi   

                                                  Tweet  \
0     @HelmiFelis

In [133]:
cleanData[['Tweet', 'Result Prediction', 'text_preprocessed']].to_csv('pemilu.csv')
  